In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from sklearn.preprocessing import LabelEncoder

In [2]:
ds = pd.read_csv('../../Machine-Learning-with-R-datasets/mushrooms.csv')
ds.head()

type cap_shape cap_surface cap_color bruises odor gill_attachment  \
0    p         x           s         n       t    p               f   
1    e         x           s         y       t    a               f   
2    e         b           s         w       t    l               f   
3    p         x           y         w       t    p               f   
4    e         x           s         g       f    n               f   

  gill_spacing gill_size gill_color   ...   stalk_surface_below_ring  \
0            c         n          k   ...                          s   
1            c         b          k   ...                          s   
2            c         b          n   ...                          s   
3            c         n          n   ...                          s   
4            w         b          k   ...                          s   

  stalk_color_above_ring stalk_color_below_ring veil_type veil_color  \
0                      w                      w         p          w   
1                      w                      w         p          w   
2                      w                      w         p          w   
3                      w                      w         p          w   
4                      w                      w         p          w   

  ring_number ring_type spore_print_color population habitat  
0           o         p                 k          s       u  
1           o         p                 n          n       g  
2           o         p                 n          n       m  
3           o         p                 k          s       u  
4           o         e                 n          a       g  

[5 rows x 23 columns]

In [4]:
le = LabelEncoder()
df = ds.apply(le.fit_transform) 
df.head()

type  cap_shape  cap_surface  cap_color  bruises  odor  gill_attachment  \
0     1          5            2          4        1     6                1   
1     0          5            2          9        1     0                1   
2     0          0            2          8        1     3                1   
3     1          5            3          8        1     6                1   
4     0          5            2          3        0     5                1   

   gill_spacing  gill_size  gill_color   ...     stalk_surface_below_ring  \
0             0          1           4   ...                            2   
1             0          0           4   ...                            2   
2             0          0           5   ...                            2   
3             0          1           5   ...                            2   
4             1          0           4   ...                            2   

   stalk_color_above_ring  stalk_color_below_ring  veil_type  veil_color  \
0                       7                       7          0           2   
1                       7                       7          0           2   
2                       7                       7          0           2   
3                       7                       7          0           2   
4                       7                       7          0           2   

   ring_number  ring_type  spore_print_color  population  habitat  
0            1          4                  2           3        5  
1            1          4                  3           2        1  
2            1          4                  3           2        3  
3            1          4                  2           3        5  
4            1          0                  3           0        1  

[5 rows x 23 columns]

In [29]:
split = int(0.9*df.values.shape[0])
train = df.loc[:split]
print type(train)
print type(df.loc)
test = df.loc[split:]
test = test.reset_index().drop('index', axis=1)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.indexing._LocIndexer'>


In [20]:
print type(df['type'])
#print df['type']
count = 0
print len(df['type'])
x = np.unique(np.array(df['type']), return_counts=True)
print x
for ix in x[1]:
    print float(ix)/x[1].sum()

<class 'pandas.core.series.Series'>
8124
(array([0, 1]), array([4208, 3916]))
0.517971442639
0.482028557361


In [25]:
def prior_probability(data, label_column, label_value):
    n_rows = data.loc[data[label_column]==label_value]
    return float(n_rows.shape[0])/data.shape[0]

def conditional_probability(data, feature_column, feature_value, label_column, label_value):
    satisfiable_constraint = data.loc[(data[label_column]==label_value) & (data[feature_column]==feature_value)]
    n_rows = data.loc[data[label_column]==label_value]
    return float(satisfiable_constraint.shape[0])/n_rows.shape[0]
print conditional_probability(train, 'cap_shape', 5, 'type', 1)

0.483107132544


In [26]:
py = prior_probability(train, 'type', 1)
print py
cp = conditional_probability(train,'cap_shape', 5, 'type', 1)
print cp

0.473605032823
0.483107132544


In [34]:
preds = []
CLASSES = [0, 1]
for ix in range(test.shape[0]):
    #print ix
    val = test.loc[ix][1:]
    feature = dict(val)
    likelihoods = []
    for cx in CLASSES:
        likelihood = 1.0
        for fx in feature:
            cp = conditional_probability(train, fx, feature[fx],'type',cx)
            likelihood *= cp
        
        prior = prior_probability(train, 'type', cx)
        
        likelihoods.append(prior*likelihood)
    
    preds.append(np.array(posterior).argmax())

In [35]:
(preds == test['type']).sum()/ float(test['type'].shape[0])

0.97539975399753998